In [1]:
#| default_exp learner

In [1]:
#|export
import math, torch, matplotlib as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter, itemgetter
from functools import partial
from copy import copy

from torch import optim
from torch.utils.data import DataLoader, default_collate
import torch.nn.functional as F

from miniai.conv import *

from fastprogress import progress_bar, master_bar

In [2]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn, tensor
from datasets import load_dataset, load_dataset_builder
from miniai.datasets import *
import logging
from fastcore.test import test_close

In [3]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [4]:
logging.disable(logging.WARNING)

##### Quick look into fascore store_attr()

`store_attr` uses `sys._getframe(1)` to get the attributes of the calling funciton. With `sys._getframe(1).f_code` to extract the name of the calling function (passing 0 for the presentframe, 1 for the previous frame, etc) and attributes with `sys._getframe(1).f_code.co_varnames`

A few examples:

In [5]:
import fastcore.all as fc

In [7]:
fc.store_attr??

In [6]:
def whoami():
    import sys
    return sys._getframe(0).f_code.co_name

In [7]:
me = whoami()

In [8]:
me

'whoami'

In [9]:
class Test():
    def __init__(self):
        print(self.whoami())
        self.ok = self.whoami()
        
    @classmethod
    def whoami(self):
        import sys
        return sys._getframe(1)#.f_code.co_name

me  = Test()

<frame at 0x55e09203d980, file '/tmp/ipykernel_107/1935095476.py', line 3, code __init__>


In [10]:
me.ok.f_code.co_varnames

('self',)

In [11]:
getattr(me.ok, 'f_code')

<code object __init__ at 0x7f09d7cda810, file "/tmp/ipykernel_107/1935095476.py", line 2>

In [12]:
me.ok.f_locals

{'self': <__main__.Test at 0x7f09d7c2b070>}

##### Details in DataLoaders and collate_dict

In [13]:
#|export
class DataLoaders:
    def __init__(self, *dls): self.train, self.valid = dls[:2]
        
    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=True):
        return cls(*[DataLoader(ds, batch_size, num_workers=4, collate_fn=collate_dict(ds)) for ds in dd.values()])

In [16]:
collate_dict??

In [14]:
from operator import itemgetter

In [15]:
d = {'a':1, 'b':2, 'c':3, 'g':10, 'u':-1}

In [16]:
feat = list(d.keys())[2:5]

In [17]:
get = itemgetter(*feat)
get(d)

(3, 10, -1)

#### Learner

In [18]:
x, y = 'image', 'label'
name = 'fashion_mnist'
dsd = load_dataset(name) # dataset dict

  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [20]:
torch.flatten(TF.to_tensor(dsd['train'][0][x])).shape

torch.Size([784])

In [21]:
bs = 1024
tds = dsd.with_transform(transformi) # transformed dataset

In [22]:
dls = DataLoaders.from_dd(tds, bs) # dataloaders
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:5]

(torch.Size([1024, 784]), tensor([9, 0, 0, 3, 0]))

In [23]:
#|export
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): fc.store_attr()
    
    def one_batch(self):
        self.xb,self.yb = to_device(self.batch)
        self.preds = model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()
                            
    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)
        
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num, self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)

    def fit(self, n_epochs):
        self.accs, self.losses, self.ns = [], [], []
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epoch = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            self.one_epoch(False)    

In [24]:
m, nh = 28*28, 50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh, 10))

In [26]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

0 True 0.7292391276041666 0.73845
0 False 0.7194779575892857 0.7408


In [27]:
torch.cuda.is_available()

True

### Metrics

In [28]:
class Metric():
    def __init__(self): 
        self.reset()
        
    def reset(self): 
        self.vals = []
        self.ns = []
        
    def add(self, inp, targ=None, n=1):
        self.last = self.calc(inp, targ)
        self.vals.append(self.last)
        self.ns.append(n)
    
    @property
    def value(self):
        ns = tensor(self.ns)
        return (tensor(self.vals)*ns).sum()/ns.sum()
    
    def calc(self, inps, targs):
        return inps

In [29]:
class Accuracy(Metric):
    def calc(self, inps, targs): return (inps==targs).float().mean()

In [30]:
class EvenCount(Metric):
    def calc(self, inp, targ=None): return 1 if inp%2==0 else 0

In [31]:
seq = [5,1,2,4,6,7,8,4,5,1,3,5,9,8,7,7,4,5,1,3,2,1,3,5,6,8,4,5,4,63,2,1,85,4,8,5,4,65,2,1,5,6,5,1,5]

counter = EvenCount()

def run(seq, metr):
    for i in seq:
        metr.add(i)

In [32]:
run(seq, counter)

In [33]:
counter.vals[:3]

[0, 0, 1]

In [34]:
x, *seq[:3] = seq

In [35]:
x, *seq[:3]

(5, 1, 2, 4)

In [36]:
len(seq)

86

In [139]:
class Capsule():
    def __init__(self, name): 
        self.name = name
        
    def __call__(self, f):
        def _f(u,x): 
            print('This is the key:')
            print(u._key)
            r = f(u,x)
            print('I am inside the method')
            return r
        return _f

In [146]:
class Builder():
    def __init__(self, seq, key):
        self.seq = seq
        self._key = key
    
    @Capsule('keygen_')
    def _decode(self, x):
#         import pdb; pdb.set_trace()
        self.val = (self.seq*self._key/x).sum()
        return (self.seq*self._key/x).sum()
    
    def status(self, x):
        print(self._decode(x))
        print(self.val)

In [147]:
build = Builder(tensor(seq), 3)

In [150]:
build.status(150)

This is the key:
3
I am inside the method
tensor(15.54)
tensor(15.54)
